In [ ]:
import sys
sys.path.append('./lib')

from DBConnection import DBConnection
from ReadJsonConfig import read_json_config
import UsefulFunctions

### example code:

``` python
# the `param_name=value` syntax is optional but useful when reading &/or reviewing the code.
UsefulFunctions.duplicate_table(
    db_source=DBConnection(read_json_config('./secrets/db_config_dev.json')),
    db_dest=DBConnection(
        read_json_config('./secrets/db_config_localhost.json'), 
        editable_environment=True
    ),
    schema_name='public', 
    table_name='inventory_forecast_store',
    truncate_table=True
)
```

In [ ]:
// code here